In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import pandas as pd
import numpy as np
import hjson
import re
import matplotlib.pyplot as plt
from fuzzywuzzy import process
from unicodedata import normalize, combining
from datetime import datetime, timedelta

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from get_dataframes import GetDataframes
from tools import import_config, import_datasets, check_titre, color
from scipy.sparse import hstack

pd.set_option("display.max_columns", None)

In [2]:
config = import_config()

In [3]:
df = import_datasets("clean_datasets/machine_learning.parquet", "parquet")
df = df.copy()

2023-11-15 15:36:59 INFO     Parquet loaded ! Importing machine_learning...


In [4]:
col = [
    "imdb_id",
    "title",
    # 'adult',
    # 'backdrop_path',
    # 'budget',
    "genres",
    "actors",
    "director",
    "keywords",
    "id",
    # 'original_language',
    # 'original_title',
    "overview",
    "popularity",
    # 'production_countries',
    "release_date",
    # 'revenue',
    # 'runtime',
    # 'spoken_languages',
    # 'status',
    # 'tagline',
    # 'video',
    "vote_average",
    "vote_count",
    "url",
    "image",
    "youtube",
    # 'poster_path',
    # 'production_companies_name',
]

In [5]:
df = df[col]

In [6]:
df.sort_values(by="title")

imdb_id                 title  \
236   tt10620868                #Alive   
3339   tt2883512                 #Chef   
2545   tt6119504          #realityhigh   
2919   tt2614684                   '71   
2001   tt1022603  (500) jours ensemble   
...          ...                   ...   
1350   tt0093565        Éclair de lune   
1140   tt0463998   Écrire pour exister   
1182  tt15789038           Élémentaire   
2990   tt6021478   Épouse-moi mon pote   
943    tt1211956               Évasion   

                                                 genres  \
236               [Horreur, Action, Aventure, Thriller]   
3339                                          [Comédie]   
2545                                          [Comédie]   
2919                  [Thriller, Action, Drame, Guerre]   
2001                          [Comédie, Drame, Romance]   
...                                                 ...   
1350                          [Comédie, Drame, Romance]   
1140                                     [Crime, Drame]   
1182  [Animation, Comédie, Familial, Fantastique, Ro...   
2990                                          [Comédie]   
943                                  [Action, Thriller]   

                                                 actors  \
236           [Yoo Ah-in, Park Shin-hye, Lee Hyun-wook]   
3339        [Jon Favreau, Sofía Vergara, Emjay Anthony]   
2545          [Nesta Cooper, Keith Powers, Alicia Sanz]   
2919       [Jack O'Connell, Sean Harris, Paul Anderson]   
2001  [Joseph Gordon-Levitt, Zooey Deschanel, Geoffr...   
...                                                 ...   
1350             [Cher, Nicolas Cage, Vincent Gardenia]   
1140       [Hilary Swank, Scott Glenn, Imelda Staunton]   
1182                       [Leah Lewis, Mamoudou Athie]   
2990  [Tarek Boudali, Philippe Lacheau, Charlotte Ga...   
943   [Sylvester Stallone, Arnold Schwarzenegger, Ji...   

                   director  \
236                [Cho Il]   
3339          [Jon Favreau]   
2545     [Fernando Lebrija]   
2919         [Yann Demange]   
2001            [Marc Webb]   
...                     ...   
1350       [Norman Jewison]   
1140  [Richard LaGravenese]   
1182           [Peter Sohn]   
2990        [Tarek Boudali]   
943       [Mikael Håfström]   

                                               keywords      id  \
236   [alone, survival, escape, drone, zombie, apart...  614696   
3339  [parent child relationship, restaurant owner, ...  212778   
2545   [nerd, high school, teenage crush, social media]  455656   
2919  [1970s, riot, northern ireland, survival, sold...  252178   
2001  [date, jealousy, fight, architect, gallery, in...   19913   
...                                                 ...     ...   
1350  [new york city, italian american, lover, full ...    2039   
1140  [black people, based on novel or book, holocau...    1646   
1182  [fire, computer animation, duringcreditsstinge...  976573   
2990    [gay marriage, pretending to be gay, gay theme]  432613   
943     [prison, prisoner, ship, prison escape, escape]  107846   

                                               overview  popularity  \
236   La propagation d'un mystérieux virus plonge un...       25.70   
3339  Carl Casper, Chef cuisinier, préfère démission...       22.99   
2545  Quand une élève brillante attire enfin l'atten...       14.87   
2919  Belfast, 1971. Tandis que le conflit dégénère ...       12.64   
2001  Tom croit encore en un amour qui transfigure, ...       37.10   
...                                                 ...         ...   
1350  Alors que son mariage avec un vieil ami approc...       12.34   
1140  Erin Gruwell, enseignante novice de 23 ans, a ...       34.23   
1182  Dans la ville d’Element City, le feu, l’eau, l...      474.38   
2990  Yassine, jeune étudiant marocain vient à Paris...        8.57   
943   Ray Breslin est un ingénieur spécialisé dans l...       27.55   

     release_date  vote_average  vote_count  \
236    2

In [7]:
col_rename = {
    "imdb_id": "titre_id",
    "id": "tmdb_id",
    "title": "titre_str",
    "genres": "titre_genres",
    "actors": "actors",
    "director": "director",
    "keywords": "keywords",
    "overview": "overview",
    "popularity": "popularity",
    "release_date": "date",
    "vote_average": "rating_avg",
    "vote_count": "rating_vote",
    "poster_path": "image",
}

In [8]:
df.rename(columns=col_rename, inplace=True)

In [9]:
df.sort_values(by="date", inplace=True)

In [10]:
df.reset_index(drop="index", inplace=True)

In [11]:
tt = [
    "actors",
    "titre_genres",
    "director",
    "keywords",
]
for t in tt:
    df[t] = df[t].apply(lambda x: ", ".join(map(str, x))).replace(" ", "")

In [12]:
df["titre_clean"] = df["titre_str"]
df["titre_clean"] = df["titre_clean"].apply(lambda x: x.lower())
df["date"] = pd.to_datetime(df["date"])
df["date"] = df["date"].dt.year

In [13]:
name = "clean_datasets/site_web.parquet"
df.reset_index(drop="index", inplace=True)
df.to_parquet(name)

In [15]:
def clean_overview(text: str) -> str:
    text = text.lower()
    text = re.sub(r"[^a-z]", " ", text)
    words = text.split()
    words = [w for w in words if w not in stopwords.words("french")]
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    return " ".join(words)

In [16]:
def supprimer_accents(texte):
    texte_clean = normalize("NFKD", texte)
    return "".join([c for c in texte_clean if not combining(c)])


tt = [
    "actors",
    "titre_genres",
    "keywords",
    "director",
    "titre_clean",
    "overview",
]
for t in tt:
    df[t] = df[t].astype(str).apply(supprimer_accents)

In [17]:
t = df[df["actors"].str.contains("Funes")]
print(t.to_markdown())

|     | titre_id   | titre_str                    | titre_genres             | actors                                         | director     | keywords                                                                                                                                   |   tmdb_id | overview                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |   popularity |   date |   rating_

In [18]:
print("cleaning overview")
df["overview"] = df["overview"].astype(str).apply(clean_overview)

cleaning overview


In [19]:
def full_lower(text: str):
    # return text.replace(" ", "").replace("-", "").lower()
    return (
        text.replace(" ", "")
        .replace("-", "")
        .replace("'", "")
        .replace(":", "")
        .lower()
    )

tt = [
    "actors",
    "titre_genres",
    "director",
    "keywords",
    "titre_clean",
]
for t in tt:
    print(f"lowering everything in {t}")
    df[t] = df[t].apply(full_lower)

lowering everything in actors
lowering everything in titre_genres
lowering everything in director
lowering everything in keywords
lowering everything in titre_clean


In [20]:
name = "clean_datasets/machine_learning_final.parquet"
df.reset_index(drop="index", inplace=True)
df.to_parquet(name)